In [1]:
import sys
import numpy as np
import cv2


import os
import scipy.misc
from scipy.optimize import least_squares
import math
from copy import deepcopy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sfm_utils import *



In [2]:
image_data_dir = '../data/statue/'
unit_test_camera_matrix = np.load('../data/unit_test_camera_matrix.npy')
unit_test_image_matches = np.load('../data/unit_test_image_matches.npy')
image_paths = [os.path.join(image_data_dir, 'images', x) for x in sorted(os.listdir('../data/statue/images')) if '.jpg' in x]

In [3]:
im0 = cv2.imread(image_paths[0])
im_height, im_width, _ = im0.shape

In [4]:
print(len(image_paths))

5


In [5]:

focal_length = 719.5459
matches_subset = np.load(os.path.join(image_data_dir,'matches_subset.npy'), allow_pickle=True,encoding='latin1')[0,:]
dense_matches = np.load(os.path.join(image_data_dir, 'dense_matches.npy'),allow_pickle=True,encoding='latin1')
fundamental_matrices = np.load(os.path.join(image_data_dir,'fundamental_matrices.npy'),allow_pickle=True,encoding='latin1')[0,:]


In [6]:
class Frame:
    def __init__(self, matches, focal_length, F, im_width, im_height):
        self.focal_length = focal_length
        self.im_height = im_height
        self.im_width = im_width
        self.matches = matches

        self.N = matches.shape[0]
        self.match_idx = np.array([np.arange(self.N), np.arange(self.N, 2 * self.N)])
        self.match_points = np.vstack((matches[:,:2], matches[:,2:]))

        self.K = np.eye(3)
        self.K[0,0] = self.K[1,1] = focal_length
        self.E = self.K.T.dot(F).dot(self.K)
        self.T = estimate_RT_from_E(self.E, matches.reshape((-1,2,2)), self.K)

        self.motion = np.zeros((2,3,4))
        self.motion[0,:,:-1] = np.eye(3)
        self.motion[1,:,:] = self.T
        self.structure = triangulate(self)

In [7]:
frames = [0] * (len(image_paths) - 1)
for i in range(len(image_paths)-1):
        frames[i] = Frame(matches_subset[i].T, focal_length,
                fundamental_matrices[i], im_width, im_height)
        bundle_adjustment(frames[i])

C:\Users\nmadali\Documents\Udemy-Reconstruction-3D\Chapitre 5\triangulation.py:197: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P_homo = np.array([P[0], P[1], P[2], 1.0])


Pour effectuer la transition d'une caméra à une troisième, en utilisant les matrices de rotation et de translation, voici les étapes :

1. **Transfert de la première caméra à la deuxième caméra** :

   Étant donné la matrice de rotation $R_{12}$ et le vecteur de translation $t_{12}$ qui décrivent la transformation de la première caméra à la deuxième caméra, vous pouvez utiliser les éléments suivants pour transformer un point $P$ du système de coordonnées de la première caméra au système de coordonnées de la deuxième caméra avec l'équation suivante :

   $$P_{2} = R_{12} \cdot P_{1} + t_{12}$$

   Ici, $P_{1}$ est la position du point dans le système de coordonnées de la première caméra, $P_{2}$ est la position du point dans le système de coordonnées de la deuxième caméra.

2. **Transfert de la deuxième caméra à la troisième caméra** :

   De même, étant donné la matrice de rotation $R_{23}$ et le vecteur de translation $t_{23}$ qui décrivent la transformation de la deuxième caméra à la troisième caméra, vous pouvez transformer le point $P_{2}$ du système de coordonnées de la deuxième caméra au système de coordonnées de la troisième caméra avec l'équation suivante :

   $$P_{3} = R_{23} \cdot P_{2} + t_{23}$$

   Ici, $P_{2}$ est la position du point dans le système de coordonnées de la deuxième caméra, $P_{3}$ est la position du point dans le système de coordonnées de la troisième caméra.

3. **Transformation combinée** :

   Si vous souhaitez directement transformer un point du système de coordonnées de la première caméra au système de coordonnées de la troisième caméra, vous pouvez combiner les transformations comme suit :

   $$P_{3} = R_{23} \cdot (R_{12} \cdot P_{1} + t_{12}) + t_{23}$$

   Cela se simplifie en :

   $$P_{3} = R_{23} \cdot R_{12} \cdot P_{1} + R_{23} \cdot t_{12} + t_{23}$$

Assurez-vous d'avoir correctement défini les matrices de rotation et les vecteurs de translation, et que les systèmes de coordonnées soient cohérents entre toutes les transformations.

* Précédemment, nous avons travaillé sur un système composé de deux caméras

<img src="https://drive.google.com/uc?export=view&id=1wNGTJyDez5xSHJ6yN89lRB8tbA-eoMdd">

* Avec une matrice de projection respective :
$$M=K [I \mid 0 ] $$ $$M'=K [R \mid t ] $$
* Par conséquent, la transition entre la deuxième et la première matrice est donnée par

$$[R^T \mid -R^Tt ]$$

* Pour fair le passage de la troisieme camera a la premiere :

   $$P = R_{12} \cdot R_{23} \cdot P_{1} + R_{12} \cdot t_{23} + t_{12}$$





In [9]:
def inverse(x):
    return np.hstack((x[:3, :3].T, -x[:3, :3].T.dot(x[:3, -1]).reshape((3,-1))))
def multiply_transformations(A, B):
    return np.hstack((A[:,:3].dot(B[:,:3]), (A[:,:3].dot(B[:,-1]) + A[:,-1]).reshape((3,-1))))

In [10]:
def transform_points(points_3d, Rt, is_inverse = False):
    if is_inverse:
        return Rt[:,:3].T.dot((points_3d - Rt[:,-1]).T).T
    return Rt[:,:3].dot(points_3d.T).T + Rt[:,-1]

In [13]:
frameA=frames[0] 
frameB=frames[1]  

In [14]:
merged_frame = deepcopy(frameA)

frameB_to_A = multiply_transformations(inverse(frameA.motion[-1,:,:]), frameB.motion[0,:,:])
frameB.structure = transform_points(frameB.structure, frameB_to_A)
for i in range(2):
        frameB.motion[i,:,:] = multiply_transformations(frameB.motion[i,:,:], inverse(frameB_to_A))

# puisque la caméra se trouve dans le cadre de référence fusionné, l'ajouter à la matrice de mouvement
merged_frame.motion = np.vstack((merged_frame.motion, frameB.motion[-1,:,:].reshape((-1,3,4))))